# Eager Execution

In [1]:
import syft as sy
sy.requires(">=0.8.1-beta")

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /home/dk/Desktop/projects/PySyft


✅ The installed version of syft==0.8.1b2 matches the requirement >=0.8.1b0


In [2]:
node = sy.orchestra.launch(name="test-domain-1", port=8080, dev_mode=True)

Starting test-domain-1 server on 0.0.0.0:8080

SQLite Store Path:
!open file:///tmp/7bca415d13ed1ec841f0d0aede098dbb.sqlite



INFO:     Started server process [130717]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('0.0.0.0', 8080): address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


In [3]:
client = node.login(email="info@openmined.org", password="changethis")
client

<SyftClient - test-domain-1 <7bca415d13ed1ec841f0d0aede098dbb>: HTTPConnection: http://localhost:8080>

In [4]:
guest_client = node.client.guest()
guest_client

<SyftClient - test-domain-1 <7bca415d13ed1ec841f0d0aede098dbb>: HTTPConnection: http://localhost:8080>

In [5]:
assert client.credentials != guest_client.credentials

# Simple permissions

In [8]:
import numpy as np

In [7]:
input_obj = sy.TwinObject(
    private_obj=np.array([[3,3,3], [3,3,3]]),
    mock_obj=np.array([[1,1,1], [1,1,1]])
)

In [8]:
# user creates
input_ptr = client.api.services.action.set(input_obj)

# guest gets pointer
pointer = guest_client.api.services.action.get_pointer(input_ptr.id)
pointer.id

<UID: 9fd6d3787e9e4b34914f362821d50478>

In [9]:
flat_ptr = pointer.flatten()
flat_ptr

```python
TwinPointer(Mock)
```
array([1, 1, 1, 1, 1, 1])

In [10]:
# read permission error
res_guest = guest_client.api.services.action.get(flat_ptr.id)
print(res_guest)
assert not isinstance(res_guest, sy.ActionObject)

Permission: <READ: e68e0f4f6cc0449cb2135421c130723a as b222cb892bfdda7f48581db464ebc7e3fcd59bc04df0b65fb144f822179021e5> denied


In [11]:
# get as user with permission
res_root = flat_ptr.get_from(client)
assert all(res_root == [3,3,3,3,3,3])

# Plans

## Plans with object instantiations

In [12]:
@sy.planify
def my_plan(x=np.array([1,2,3,4,5,6])):
    return x+1

In [13]:
plan_ptr = my_plan.send(guest_client)

In [14]:
input_obj = sy.TwinObject(
    private_obj=np.array([1,2,3,4,5,6]),
    mock_obj=np.array([1,1,1,1,1,1])
)

In [15]:
_id = client.api.services.action.set(input_obj).id
pointer = guest_client.api.services.action.get_pointer(_id) 

In [16]:
res_ptr = plan_ptr(x=pointer)

In [17]:
res_ptr.get_from(client)

array([2, 3, 4, 5, 6, 7])

## Complex plans

In [18]:
@sy.planify
def my_plan(x=np.array([1,2,3,4,5,6])):
    y = x.flatten() # method -> [1,2,3,4,5,6]
    min_val = x.min() # method -> 1
    s = x.shape # getattribute -> 6    
    w = x[min_val] # __getitem__ -> 2
    y[min_val] = min_val # __setitem__ y = [1,1,3,4,5,6]
    res = client.api.lib.numpy.sum(y) + w + s # client function and __add__ 20 + 2 + 6 = 28
    return res #28

In [19]:
my_plan

```python
Pointer
```
```python
class Plan:
  id: str = e870208895c64d7ba64cabf6421d1b56
  inputs: str = {'x': Pointer:
[1 2 3 4 5 6]}
  outputs: str = [Pointer:
[28]]
  actions: str = [ActionObject NumpyArrayObject[f58..8].flatten(,), ActionObject NumpyArrayObject[f58..8].min(,), ActionObject NumpyArrayObject[f58..8].shape(,), ActionObject NumpyArrayObject[f58..8].__getitem__(104..a,), ActionObject NumpyArrayObject[083..9].__setitem__(104..a, 104..a,), ActionObject numpy.sum(083..9,), ActionObject NumpyScalarObject[48f..f].__add__(6a3..4,), ActionObject NumpyScalarObject[473..e].__add__(c91..8,)]
  code: str = "@sy.planify
def my_plan(x=np.array([1,2,3,4,5,6])):
    y = x.flatten() # method -> [1,2,3,4,5,6]
    min_val = x.min() # method -> 1
    s = x.shape # getattribute -> 6    
    w = x[min_val] # __getitem__ -> 2
    y[min_val] = min_val # __setitem__ y = [1,1,3,4,5,6]
    res = client.api.lib.numpy.sum(y) + w + s # client function and __add__ 20 + 2 + 6 = 28
    return res #28
"
  client: str = None
  syft_passthrough_attrs: str = ['inputs', 'outputs', 'code', 'actions', 'client']

```

In [20]:
plan_ptr = my_plan.send(guest_client)

In [21]:
input_obj = sy.TwinObject(
    private_obj=np.array([1,2,3,4,5,6]),
    mock_obj=np.array([1,1,1, 1,1,1])
)

_id = client.api.services.action.set(input_obj).id
pointer = guest_client.api.services.action.get_pointer(_id) 

In [22]:
res_ptr = plan_ptr(x=pointer)

In [23]:
res_ptr.get_from(client)

array([29])

## Plans with function calls

In [24]:
client.api.lib.numpy.sum

<function syft.client.api.generate_remote_lib_function.<locals>.wrapper(*args, **kwargs)>

In [25]:
@sy.planify
def my_plan(x=np.array([[2,2,2], [2,2,2]])):
    y = x.flatten()
    w = client.api.lib.numpy.sum(y)
    return w

In [26]:
plan_ptr = my_plan.send(guest_client)

input_obj = sy.TwinObject(
    private_obj=np.array([[3,3,3], [3,3,3]]),
    mock_obj=np.array([[1,1,1], [1,1,1]])
)

input_ptr = client.api.services.action.set(input_obj)
pointer = guest_client.api.services.action.get_pointer(input_ptr.id) 


In [27]:
res_ptr = plan_ptr(x=pointer)

In [28]:
assert res_ptr.get_from(client) == 18

## simple execution (root downloads)

In [9]:
@sy.planify
def my_plan(x=np.array([[2,2,2], [2,2,2]])):
    y = x.flatten()
    z = y.prod()
    return z

In [10]:
plan_ptr = my_plan.send(guest_client)

In [11]:
input_obj = sy.TwinObject(
    private_obj=np.array([[3,3,3], [3,3,3]]),
    mock_obj=np.array([[1,1,1], [1,1,1]])
)

In [12]:
input_ptr = client.api.services.action.set(input_obj)

In [13]:
pointer = guest_client.api.services.action.get_pointer(input_ptr.id) 

In [14]:
res_ptr = plan_ptr(x=pointer)

In [15]:
# TODO: should be 1
res_ptr

```python
Pointer
```
64

In [16]:
assert not isinstance(guest_client.api.services.action.get(res_ptr.id), sy.ActionObject)

In [17]:
assert res_ptr.get_from(client) == np.array([[3,3,3], [3,3,3]]).flatten().prod()

## Downloading the result

In [18]:
res_ptr.request(guest_client)

```python
class Request:
  id: str = a875bb35883f48e6beb2bd7e90046434
  requesting_user_verify_key: str = 7510942d230ec829959dfe97c2749b7d5faecf9a50c22c0f8b085d4fac5feafc
  approving_user_verify_key: str = None
  request_time: str = 2023-05-16 02:51:17
  approval_time: str = None
  status: str = RequestStatus.PENDING
  node_uid: str = 7bca415d13ed1ec841f0d0aede098dbb
  request_hash: str = "dab0850dfae86b447783e47a25642a1cf3438da0178edbbe2f6c60be3daee695"
  changes: str = [syft.service.request.request.ActionStoreChange]

```

In [26]:
# TODO: fix this issue with custom 
client.api.services.messages[2].linked_obj

```python
class LinkedObject:
  id: str = aed10d0bc56444dda9293533c8dd845d
  node_uid: str = 7bca415d13ed1ec841f0d0aede098dbb
  service_type: str = <class 'syft.service.request.request_service.RequestService'>
  object_type: str = <class 'syft.service.request.request.Request'>
  object_uid: str = bc4a0a5b30f3447abb3e71ab2cea56ad

```

In [20]:
client.api.services.request[0].approve_with_client(client)

Exception: syft.user.RepeatedCallPolicy_aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f_ba90eef16c2b39292a2ee07f1aefc157216b84487b26c32a1512d114c8ee7c69 not in TYPE_BANK

In [69]:
res_ptr.get_from(guest_client)

Err('Could not find item with uid 97dd6d99029e43fea0473685e1f72c1a, "97dd6d99029e43fea0473685e1f72c1a not in <class \'syft.store.sqlite_document_store.SQLiteBackingStore\'>"')

# Pointers (Twins)

## setattribute

In [41]:
private_data, mock_data = np.array(
    [[1.0,2.0,3.0], [4.0,5.0,6.0]]), np.array([[1.0,1.0,1.0], [1.0,1.0,1.0]]
)

In [42]:
obj = sy.TwinObject(
    private_obj=private_data,
    mock_obj=mock_data
)

In [43]:
obj_pointer = client.api.services.action.set(obj)
obj_pointer = guest_client.api.services.action.get_pointer(obj_pointer.id)

In [44]:
original_id = obj_pointer.id

In [45]:
obj_pointer

```python
TwinPointer(Mock)
```
array([[1., 1., 1.],
       [1., 1., 1.]])

In [46]:
# note that this messes up the data and the shape of the array
obj_pointer.dtype = np.int32  

In [47]:
res = obj_pointer.get_from(client)

In [48]:
assert res.dtype == np.int32

In [49]:
private_data.dtype= np.int32
mock_data.dtype= np.int32

In [50]:
assert (res == private_data).all()
assert (obj_pointer.syft_action_data == mock_data).all()
assert not (obj_pointer.syft_action_data == private_data).all()

## getattribute

In [51]:
obj = sy.TwinObject(
    private_obj=np.array([[1,2,3], [4,5,6]]),
    mock_obj=np.array([[1,1,1], [1,1,1]])
)

In [52]:
obj_pointer = client.api.services.action.set(obj)
obj_pointer = guest_client.api.services.action.get_pointer(obj_pointer.id)

In [53]:
size_pointer = obj_pointer.size   

In [54]:
assert size_pointer.get_from(client) == 6

## methods

In [55]:
obj = sy.TwinObject(
    private_obj=np.array([[1,2,3], [4,5,6]]),
    mock_obj=np.array([[1,1,1], [1,1,1]])
)

In [56]:
obj_pointer = client.api.services.action.set(obj)
obj_pointer = guest_client.api.services.action.get_pointer(obj_pointer.id)

In [57]:
flat_pointer = obj_pointer.flatten()   

In [58]:
assert all(flat_pointer.get_from(client) == np.array([1,2,3,4,5,6]))

## Indexing

In [59]:
obj = sy.TwinObject(
    private_obj=np.array([[1,2,3], [4,5,6]]),
    mock_obj=np.array([[1,1,1], [1,1,1]])
)

In [60]:
obj_pointer = client.api.services.action.set(obj)
obj_pointer = guest_client.api.services.action.get_pointer(obj_pointer.id)

In [61]:
indexed_pointer = obj_pointer[0, 0:2]   

In [62]:
assert all(indexed_pointer.get_from(client) == np.array([1,2]))

In [63]:
# if node.node_type.value == "python":
node.land()

Stopping test-domain-1
